In [1]:
import numpy as np
from power import *
from power.systems.b6l8_eolic import B6L8EOL
from opf_linear.opf_loss import LinearDispatch
from opf_linear.utils.extr_and_save import extract_and_save_results
from pathlib import Path 

# Sistema B6L8

Como o sistema não possui geração eólica, o estudo consistirá na variação de carga.

## Carregar Sistema E Parâmetros de Simulação

In [2]:
net = B6L8EOL()
seed = 42
rng = np.random.default_rng(seed=seed)
nscen = 100

# 2. ✅ Definir e criar a pasta de resultados
results_dir = Path("results_b6l8")  # Cria um objeto de caminho para a pasta 'results'
results_dir.mkdir(exist_ok=True) # Cria a pasta. Se já existir, não faz nada e não dá erro.

## Geradores de Cenários

In [3]:
def apply_load_scen(net=Network, ag=0.95, bg=1.10, ac=-0.02, bc=0.02, rng=np.random.default_rng(seed=42)):
    num_loads = len(net.loads)
    base_loads_p = np.array([load.p for load in net.loads])
    rg = ag + (bg - ag) * rng.random(num_loads)  
    rc = ac + (bc - ac) * rng.random(num_loads) 

    # Calcular o fator de variação total 'r' somando os dois fatores 
    r = rg + rc
    scenario_loads = base_loads_p * r

    for idx, load_object in enumerate(net.loads):
        load_object.p = scenario_loads[idx]

In [4]:
def apply_wnd_scen(net=Network, rng=np.random.default_rng(seed=42)):
    wnd_genrators = [g for g in net.generators if isinstance(g, WindGenerator)]
    num_wnd = len(wnd_genrators)
    if not wnd_genrators:
        return np.array([])
    base_wnd_p_max = np.array([g.p_max for g in wnd_genrators])
    random_factors = rng.random(num_wnd)
    scenario_wnd_p = base_wnd_p_max * random_factors

    for idx, wnd_object in enumerate(wnd_genrators):
        wnd_object.p_max = scenario_wnd_p[idx]

## Rodando a otimização em 100 cenários

In [5]:
for scen in range(nscen):
    net = B6L8EOL()
    apply_wnd_scen(net=net, rng=rng)
    apply_load_scen(net=net, rng=rng)

    solver = LinearDispatch(net)
    status, _, _ = solver.solve_loss()
    if status == 'Optimal':
        output_filename = results_dir/f"cenario_{scen}.json"
        extract_and_save_results(solver.problem, solver.net, output_filename)
    else:
        print("DEU RUIM")

Iniciando Despacho Econômico Linear com Perdas...
Iteração 1: Perdas Totais = 0.001680 pu | Diferença = 0.001680
Iteração 2: Perdas Totais = 0.001675 pu | Diferença = 0.000005

Convergência atingida na iteração 2!

[SUCESSO] Resultados exportados para o arquivo 'results_b6l8\cenario_0.json'
Iniciando Despacho Econômico Linear com Perdas...
Iteração 1: Perdas Totais = 0.002480 pu | Diferença = 0.002480
Iteração 2: Perdas Totais = 0.002477 pu | Diferença = 0.000004

Convergência atingida na iteração 2!

[SUCESSO] Resultados exportados para o arquivo 'results_b6l8\cenario_1.json'
Iniciando Despacho Econômico Linear com Perdas...
Iteração 1: Perdas Totais = 0.002455 pu | Diferença = 0.002455
Iteração 2: Perdas Totais = 0.002463 pu | Diferença = 0.000008

Convergência atingida na iteração 2!

[SUCESSO] Resultados exportados para o arquivo 'results_b6l8\cenario_2.json'
Iniciando Despacho Econômico Linear com Perdas...
Iteração 1: Perdas Totais = 0.001208 pu | Diferença = 0.001208
Iteração 2: